# Sandbox for new features (especially to avoid conflicts)

In [16]:
from nltk.corpus import wordnet
import pandas as pd
import numpy as np

# FROM TERMINAL:
# pip install -U spacy
# python -m spacy download en
import spacy

In [3]:
def treebank_to_wn_tag(tb_tag):
    tag_dict = {'N': 'n', 'V': 'v', 'J': 'a', 'R': 'r'}
    if tb_tag[0] in tag_dict:
        return tag_dict[tb_tag[0]]
    else:
        return None

## If a candidate word isn't in the vocabulary, get some hypernyms
Should first check for synonyms in the vocabulary!

In [15]:
def get_hypernyms(word, pos=None):
    # TODO: POS tagging when choosing a synset
    # TODO: find a better way to choose a synset... but there might not be that many for these words anyway
    # TODO: error handling - what do we do if a word doesn't have a synset?
    hypernym_words = []
    for syn in wordnet.synsets(word, pos):
        print ("Synset: {}".format(syn))
        for h in syn.hypernyms():
            print ("Hypernym synset: {}".format(h))
            for l in h.lemmas():
                hypernym_words.append(l.name())
    
    print (hypernym_words)
            
        

**'voluptuous' has many synonyms, but no hypernyms**

In [16]:
get_hypernyms('happy') 

Synset: Synset('happy.a.01')
Synset: Synset('felicitous.s.02')
Synset: Synset('glad.s.02')
Synset: Synset('happy.s.04')
[]


In [11]:
def get_synonyms(word, pos=None):
    synonyms = []
    for syn in wordnet.synsets(word, pos):
        print(syn)
        for l in syn.lemmas():
            print("-", l)
            synonyms.append(l.name())
    # if not synonyms:
    #     print("No synonyms found for '{}' with pos={}".format(word, pos))
    return synonyms

In [13]:
get_synonyms("happy")

Synset('happy.a.01')
- Lemma('happy.a.01.happy')
Synset('felicitous.s.02')
- Lemma('felicitous.s.02.felicitous')
- Lemma('felicitous.s.02.happy')
Synset('glad.s.02')
- Lemma('glad.s.02.glad')
- Lemma('glad.s.02.happy')
Synset('happy.s.04')
- Lemma('happy.s.04.happy')
- Lemma('happy.s.04.well-chosen')


['happy', 'felicitous', 'happy', 'glad', 'happy', 'happy', 'well-chosen']

**'trifling' has many synsets and hypernyms if no POS tag, but if the correct POS tag given, is much more accurate**

In [6]:
get_hypernyms('trifling') 

print("\n\n")

get_hypernyms('trifling', 'a')

Synset: Synset('dalliance.n.01')
Hypernym synset: Synset('delay.n.02')
Synset: Synset('piddle.v.01')
Hypernym synset: Synset('spend.v.02')
Synset: Synset('frivol.v.01')
Hypernym synset: Synset('act.v.02')
Synset: Synset('dally.v.04')
Hypernym synset: Synset('consider.v.03')
Synset: Synset('negligible.s.02')
['delay', 'holdup', 'spend', 'expend', 'drop', 'act', 'behave', 'do', 'consider', 'take', 'deal', 'look_at']



Synset: Synset('negligible.s.02')
[]


## POS tagging, dependency parsing with spacy
[Documentation for dependency parsing](https://spacy.io/usage/linguistic-features#dependency-parse)

[Code example for dependency parsing](https://github.com/explosion/spacy/blob/master/examples/information_extraction/parse_subtrees.py)

In [7]:
nlp = spacy.load('en_core_web_sm')

In [14]:
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')

for token in doc:
    print(token.text, token.norm_, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Apple apple apple PROPN NNP nsubj Xxxxx True False
is is be VERB VBZ aux xx True True
looking looking look VERB VBG ROOT xxxx True False
at at at ADP IN prep xx True True
buying buying buy VERB VBG pcomp xxxx True False
U.K. u.k. u.k. PROPN NNP compound X.X. False False
startup startup startup NOUN NN dobj xxxx True False
for for for ADP IN prep xxx True True
$ $ $ SYM $ quantmod $ False False
1 1 1 NUM CD compound d False False
billion billion billion NUM CD pobj xxxx True False


In [9]:
def get_spacy_doc(sentence, spacy_nlp=nlp):
    return spacy_nlp(sentence)

def get_pos_of_word(spacy_doc, token_index):
    # NOTE: token index MIGHT NOT be the index of the word in the original sentence (tokenization)
    return spacy_doc[token_index].pos_

def get_pos_of_sentence(sentence):
    doc = get_spacy_doc(sentence):
    pos_list = []
    i = 0
    for token in doc:
        pos = token.pos_
        i += 1
        
        


def get_ancestors_of_word(spacy_doc, token_index):
    # NOTE: token index MIGHT NOT be the index of the word in the original sentence (tokenization)
    # NOTE: returns token OBJECTS, not just text
    token = spacy_doc[token_index]
    
    return [ancestor for ancestor in token.ancestors]


def get_children_of_word(spacy_doc, token_index):
    # NOTE: token index MIGHT NOT be the index of the word in the original sentence (tokenization)
    # NOTE: returns token OBJECTS, not just text
    token = spacy_doc[token_index]
    
    return [child for child in token.children]
        

In [10]:
sentence = "Even though they only had two days left, the _____ weren't worried at all about their assignment."
doc = get_spacy_doc(sentence, nlp)
for i, token in enumerate(doc):
    print(i, token)

0 Even
1 though
2 they
3 only
4 had
5 two
6 days
7 left
8 ,
9 the
10 _
11 _
12 _
13 _
14 _
15 were
16 n't
17 worried
18 at
19 all
20 about
21 their
22 assignment
23 .


In [10]:
token_index = 4
print(get_pos_of_word(doc, token_index))
print(get_ancestors_of_word(doc, token_index))
print(get_children_of_word(doc, token_index))

VERB
[were]
[Even, though, they, only, left]


In [11]:
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True)

## Stopword removal
The Woods paper mentions (for the MSR task) removing determiners, coordinating conjunctions, pronouns, and proper nouns for some (?) feature sets (it's kind of confusing what their actual feature sets are for the PMI model)

In [3]:
df1 = pd.read_csv('../data/Testing/combined.gzip', index_col=0, compression='gzip')

In [5]:
df1

,0,1,1-0,1-1,10,10-0,10-1,10-2,10-2-1,10-7,...,zeitgeist,zero,zigzagging,zivakovic,zolotic,zone,zoo,zoos,zucchini,zupanja
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-2-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df2 = pd.read_csv('../data/Testing/sample_coo1', index_col=0, compression='gzip')
df2

In [9]:
df3 = pd.read_csv('../data/Testing/sample_coo2', index_col=0, compression='gzip')
df3

,the,of,gonna,and,to,in,he,that,was,for,...,profit-making,entity,restore,involve,fees,recordings,pensions,length,92nd,events
the,317,466,216,275,292,346,124,128,158,125,...,1,1,2,0,1,1,3,3,0,1
of,466,46,174,134,66,106,45,42,40,43,...,0,0,1,0,0,0,1,1,1,1
gonna,216,174,48,94,101,111,49,62,67,65,...,1,1,0,0,0,0,0,0,1,1
and,275,134,94,26,97,100,39,27,39,43,...,1,0,0,0,0,0,1,1,0,0
to,292,66,101,97,29,70,51,35,44,41,...,0,0,2,0,0,0,0,0,0,0
in,346,106,111,100,70,39,46,53,51,38,...,0,0,0,0,0,0,0,0,0,1
he,124,45,49,39,51,46,11,26,54,14,...,0,0,0,0,0,0,0,0,0,0
that,128,42,62,27,35,53,26,4,21,13,...,0,0,0,0,0,0,0,0,0,0
was,158,40,67,39,44,51,54,21,5,11,...,0,0,0,0,0,0,0,0,0,0
for,125,43,65,43,41,38,14,13,11,7,...,0,0,0,1,1,1,0,0,0,0


In [5]:
df4 = pd.read_csv('../data/Testing/sample_context_coo2', index_col=0, compression='gzip')
df4

,the,a,of,and,to,in,he,that,was,for,...,profit-making,entity,restore,involve,fees,recordings,pensions,length,92nd,events
the,1289,941,1219,907,880,1025,330,402,364,397,...,1,1,4,3,3,3,3,3,4,4
a,941,223,489,344,348,380,144,172,156,171,...,1,1,1,0,0,0,0,0,1,1
of,1219,489,249,428,283,430,143,148,137,152,...,0,0,1,1,1,1,1,1,2,2
and,907,344,428,142,299,345,144,125,134,138,...,2,2,0,1,1,1,1,1,0,0
to,880,348,283,299,144,253,125,127,122,129,...,0,0,2,0,0,0,0,0,1,1
in,1025,380,430,345,253,181,136,146,139,127,...,0,0,1,0,0,0,0,0,1,1
he,330,144,143,144,125,136,43,65,109,43,...,0,0,0,0,0,0,0,0,0,0
that,402,172,148,125,127,146,65,28,62,53,...,1,1,0,0,0,0,0,0,0,0
was,364,156,137,134,122,139,109,62,33,44,...,0,0,0,0,0,0,0,0,1,1
for,397,171,152,138,129,127,43,53,44,30,...,0,0,0,1,1,1,1,1,0,0


In [8]:
df4 = pd.read_csv('../data/GIGA/batch_01_ww_15000_4.csv.gz', index_col=0, compression='gzip')
df4

,the,to,of,gonna,and,in,that,for,is,on,...,nokia,camel,flap,oversaw,depleted,bracket,baton,compromises,scoreboard,lancaster
the,560322,640391,1037331,369566,534634,640394,274937,273418,270570,240283,...,17,61,135,231,74,136,91,59,225,92
to,640391,98297,186123,250693,214724,163585,108366,91189,104696,69976,...,3,25,39,19,17,29,34,72,33,36
of,1037331,186123,81509,315196,245826,201514,95580,88919,97432,70261,...,7,36,27,104,29,24,25,25,28,58
gonna,369566,250693,315196,77859,197962,229193,112527,121752,118869,80853,...,2,73,46,33,14,26,34,29,50,55
and,534634,214724,245826,197962,64281,189485,78808,92660,68556,67703,...,14,60,19,45,44,53,46,38,39,53
in,640394,163585,201514,229193,189485,51715,73741,63688,67830,41734,...,1,24,33,25,18,58,43,21,38,53
that,274937,108366,95580,112527,78808,73741,10209,31594,75462,27995,...,0,20,23,13,14,13,5,20,18,4
for,273418,91189,88919,121752,92660,63688,31594,11926,40468,22914,...,4,23,10,18,5,28,7,6,8,11
is,270570,104696,97432,118869,68556,67830,75462,40468,12261,24269,...,0,12,21,0,16,7,6,11,5,17
on,240283,69976,70261,80853,67703,41734,27995,22914,24269,6469,...,1,9,15,5,3,10,9,15,52,6


In [52]:
df1 = pd.read_csv('../data/Testing/sample_coo1', index_col=0, compression='gzip')
df2 = pd.read_csv('../data/Testing/sample_coo2', index_col=0, compression='gzip')

In [51]:
print(df1.head())
print("")
print(df2.head())

       the  to  of  and  gonna  in  it  time  for  with  ...    wire  racks  \
the     12  19  24   23     13  23   6     7   10    14  ...       0      0   
to      19   2   5    8      9   3   2     3    3     2  ...       0      0   
of      24   5   4   12     13   5   1     5    0     1  ...       0      0   
and     23   8  12    2      4   8   4     4    0     3  ...       1      1   
gonna   13   9  13    4      2   4   4     3    6     3  ...       0      0   

       spray  mace  bring  guests  comatose  state  able  solid  
the        1     1      0       0         0      0     0      0  
to         1     1      1       1         1      1     1      1  
of         0     0      0       1         1      1     1      0  
and        0     0      0       0         1      1     1      1  
gonna      0     0      0       1         1      1     1      0  

[5 rows x 737 columns]

       the   of  gonna  and   to   in   he  that  was  for   ...    \
the    317  466    216  275  292  

In [57]:
combined = combine_cooccurrence_matrices(df1, df2)

Creating set of vocab words for both matrices
Beginning work on combining matrices

Done with df 0 columns
Size of augmented matrix 0: 5507 by 5507
Starting index sort
Done with df 0

Done with df 1 columns
Size of augmented matrix 1: 5507 by 5507
Starting index sort
Done with df 1

Size of augmented matrices: 
5507 5507
5507 5507

Sorting by word frequency


In [48]:
combined.head()
# combined.isnull().values.any()

,the,of,gonna,and,to,in,he,was,that,for,...,recorders,sauce,seafood,slipped,solid,sympathetic,tate,teaspoon,tipsy,tow
the,329.0,490.0,229.0,298.0,311.0,369.0,125.0,163.0,132.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
of,490.0,50.0,187.0,146.0,71.0,111.0,46.0,41.0,43.0,43.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
gonna,229.0,187.0,50.0,98.0,110.0,115.0,49.0,70.0,63.0,71.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
and,298.0,146.0,98.0,28.0,105.0,108.0,39.0,40.0,31.0,43.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
to,311.0,71.0,110.0,105.0,31.0,73.0,51.0,45.0,38.0,44.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [58]:
combined_df = pd.read_csv('../data/Testing/sample_combined', index_col=0, compression='gzip')


In [59]:
combined_df.head()

,the,of,gonna,and,to,in,he,was,that,for,...,recorders,sauce,seafood,slipped,solid,sympathetic,tate,teaspoon,tipsy,tow
the,329.0,490.0,229.0,298.0,311.0,369.0,125.0,163.0,132.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
of,490.0,50.0,187.0,146.0,71.0,111.0,46.0,41.0,43.0,43.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
gonna,229.0,187.0,50.0,98.0,110.0,115.0,49.0,70.0,63.0,71.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
and,298.0,146.0,98.0,28.0,105.0,108.0,39.0,40.0,31.0,43.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
to,311.0,71.0,110.0,105.0,31.0,73.0,51.0,45.0,38.0,44.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
